In [9]:

import csv
import pandas as pd
import matplotlib.pyplot as plt

In [10]:
caproviderdata = pd.read_csv("Enrolled_Medi-Cal_Fee_For_Service_Provider_File.csv", low_memory=False)
caprovider_df = pd.DataFrame(caproviderdata)
caprovider_df.dropna(inplace=True)
caprovider_df.reset_index(drop=True, inplace=True) 

#YOU GOT THIS WARNING, SO I LEFT THIS CONVERSION (AFTER DROPPING NULLS I FOUND WHAT'S LEFT IS JUST NUMERIC, NO MORE ALPHA):
#C:\Users\Kdw86\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (2) have mixed types.
#Specify dtype option on import or set low_memory=False.
# has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
#caprovider_df['Provider_Number'] = caprovider_df['Provider_Number'].astype('int64')

for i, row in caprovider_df.iterrows():
    try:
        row['Provider_Number'] = int(row['Provider_Number'])
    except:
        caprovider_df.drop[i, :]

#len(caprovider_df['Provider_Number'])        
caprovider_df.head()

,X,Y,Provider_Number,NPI_Provider_Number,Owner_Number,Service_Location_Number,Provider_Legal_Name,Enrollment_Status_Effective_Dat,Provider_Type_Code,Provider_Type_Code_Desc,...,Provider_Address_Line_2,Provider_Address_City,Provider_Address_State,Provider_Address_Zip,Provider_Address_Latitude,Provider_Address_Longitude,MSSA_ID,Tract_ID,File_,ObjectId
